# 模型训练

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/beginner/mindspore_train.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/beginner/mindspore_train.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/beginner/train.ipynb)

模型训练一般分为四个步骤：

1. 构建数据集。
2. 定义神经网络。
3. 定义超参、损失函数及优化器。
4. 输入数据集进行训练与评估。

通过上面章节的学习，我们已经学会如何构建数据集和创建模型，本章将主要介绍如何设置超参和训练模型。本章以MNIST数据集和LeNet网络为例，介绍使用MNIST数据集训练LeNet网络。

从数据处理和创建网络章节中加载先前代码。

In [1]:
import os
import requests
import mindspore.dataset as ds
import mindspore.dataset.vision as vision
import mindspore.nn as nn
from mindspore.common.initializer import Normal

requests.packages.urllib3.disable_warnings()


def download_dataset(dataset_url, path):
    filename = dataset_url.split("/")[-1]
    save_path = os.path.join(path, filename)
    if os.path.exists(save_path):
        return
    if not os.path.exists(path):
        os.makedirs(path)
    res = requests.get(dataset_url, stream=True, verify=False)
    with open(save_path, "wb") as f:
        for chunk in res.iter_content(chunk_size=512):
            if chunk:
                f.write(chunk)
    print("The {} file is downloaded and saved in the path {} after processing".format(os.path.basename(dataset_url),
                                                                                       path))


train_path = "datasets/MNIST_Data/train"
test_path = "datasets/MNIST_Data/test"

# 下载数据集
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-labels-idx1-ubyte",
                 train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-images-idx3-ubyte",
                 train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-labels-idx1-ubyte",
                 test_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-images-idx3-ubyte",
                 test_path)


def create_dataset(data_path, batch_size=16, image_size=32):
    # 加载MINIST数据集
    dataset = ds.MnistDataset(data_path)

    rescale = 1.0 / 255.0
    shift = 0.0

    # 图像增强操作
    trans = [
        vision.Resize(size=image_size),
        vision.Rescale(rescale, shift),
        vision.HWC2CHW(),
    ]
    dataset = dataset.map(operations=trans, input_columns=["image"])
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset


# 自定义网络
class LeNet(nn.Cell):
    def __init__(self, num_classes=10, num_channel=1):
        super(LeNet, self).__init__()
        layers = [nn.Conv2d(num_channel, 6, 5, pad_mode='valid'),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Conv2d(6, 16, 5, pad_mode='valid'),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Flatten(),
                  nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02)),
                  nn.ReLU(),
                  nn.Dense(120, 84, weight_init=Normal(0.02)),
                  nn.ReLU(),
                  nn.Dense(84, num_classes, weight_init=Normal(0.02))]
        # 使用CellList对网络进行管理
        self.build_block = nn.CellList(layers)

    def construct(self, x):
        # for循环执行网络
        for layer in self.build_block:
            x = layer(x)
        return x


# 定义神经网络
network = LeNet()

## 超参（Hyper-parametric）

超参是可以调整的参数，可以控制模型训练优化的过程，不同的超参数值可能会影响模型训练和收敛速度。目前深度学习模型多采用批量随机梯度下降算法进行优化，随机梯度下降算法的原理如下：

$$w_{t+1}=w_{t}-\eta \frac{1}{n} \sum_{x \in \mathcal{B}} \nabla l\left(x, w_{t}\right)$$

式中，$n$是批量大小(batch size)，$η$是学习率(learning rate)；另外，$w_{t}$为训练轮次t中权重参数，$\nabla l$为损失函数的导数。可知道除了梯度本身，这两个因子直接决定了模型的权重更新，从优化本身来看它们是影响模型性能收敛最重要的参数。一般会定义以下超参用于训练：

训练轮次（epoch）：训练时遍历数据集的次数。

批次大小（batch size）：数据集进行分批读取训练，设定每个批次数据的大小。batch size过小，花费时间多，同时梯度震荡严重，不利于收敛；batch size过大，不同batch的梯度方向没有任何变化，容易陷入局部极小值，因此需要选择合适的batch size，可以有效提高模型精度、全局收敛。

学习率（learning rate）：如果学习率偏小，会导致收敛的速度变慢，如果学习率偏大则可能会导致训练不收敛等不可预测的结果。梯度下降法是一个广泛被用来最小化模型误差的参数优化算法。梯度下降法通过多次迭代，并在每一步中最小化损失函数来估计模型的参数。学习率就是在迭代过程中，会控制模型的学习进度。

In [2]:
epochs = 10
batch_size = 32
learning_rate = 1e-2

## 定义损失函数

**损失函数**用来评价模型的**预测值**和**目标值**之间的误差，在这里，使用`SoftmaxCrossEntropyWithLogits`计算预测值与真实值之间的交叉熵。代码用例如下：

In [3]:
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

## 定义优化器

**优化器函数**用于计算和更新梯度，模型优化算法的选择直接关系到最终模型的性能。有时候最终模型效果不好，未必是特征或者模型设计的问题，很有可能是优化算法的问题。在这里，我们使用`Momentum`优化器。`mindspore.nn`也提供了许多其他常用的优化器函数，如`Adam`、`SGD`、`RMSProp`等。

我们需要构建一个Optimizer对象，这个对象能够基于计算得到的梯度对参数进行更新。为了构建一个Optimizer，需要给它一个包含可优化的参数，如网络中所有可以训练的parameter，即设置优化器的入参为`network.trainable_params()`。

然后，设置Optimizer的参数选项，比如学习率、权重衰减等。代码样例如下：

In [4]:
net_opt = nn.Momentum(network.trainable_params(), learning_rate=learning_rate, momentum=0.9)

## 定义训练流程

首先定义前向网络`forward`，输出正向传播网络的损失值；然后定义单个step训练流程`train_step`，通过反向传播实现网络权重的更新。最后定义训练流程`train_epoch`，传入训练数据集和模型，实现单个epoch的训练。

In [5]:
import mindspore as ms
import mindspore.ops as ops


# 前向网络
def forward(inputs, targets):
    outputs = network(inputs)
    loss = net_loss(outputs, targets)
    return loss


grad_fn = ops.value_and_grad(forward, None, net_opt.parameters)


# 定义单个step训练流程，实现网络权重的更新
def train_step(inputs, targets):
    loss, grads = grad_fn(inputs, targets)
    net_opt(grads)
    return loss


# 定义单个epoch训练流程
def train_epoch(dataset, network):
    # 设置网络为训练模式
    network.set_train()
    # 使用数据集对网络进行训练
    for data in dataset.create_dict_iterator():
        loss = train_step(ms.Tensor(data['image'], ms.float32), ms.Tensor(data["label"], ms.int32))

    return loss

## 定义验证流程

定义验证流程`evaluate_epoch`，传入待验证的数据集和训练好的网络，计算网络分类的准确率。

In [6]:
import numpy as np


def evaluate_epoch(dataset, network):
    size = dataset.get_dataset_size()
    accuracy = 0

    for data in dataset.create_dict_iterator():
        output = network(ms.Tensor(data['image'], ms.float32))
        acc = np.equal(output.argmax(1).asnumpy(), data["label"].asnumpy())
        accuracy += np.mean(acc)
    accuracy /= size

    return accuracy

## 模型训练与评估

定义好训练流程和验证流程后，加载训练数据集和验证数据集后，即可进行模型的训练与评估，

In [7]:
import mindspore as ms
import mindspore.ops as ops
import time

# 加载训练数据集
dataset_train = create_dataset(train_path, batch_size)
# 加载测试数据集
dataset_test = create_dataset(test_path, batch_size)

for epoch in range(epochs):
    start = time.time()
    result = train_epoch(dataset_train, network)
    acc = evaluate_epoch(dataset_test, network)
    end = time.time()
    time_ = round(end - start, 2)

    print("-" * 20)
    print(f"Epoch:[{epoch}/{epochs}], "
          f"Train loss:{result.asnumpy():5.4f}, "
          f"Accuracy:{acc:5.3f}, "
          f"Time:{time_}s, ")

[EVENT] (PID29692) 2022-09-08-10:55:46.568.557 Start compiling 'after_grad' and it will take a while. Please wait...
[EVENT] (PID29692) 2022-09-08-10:55:46.695.443 End compiling 'after_grad'.
[EVENT] (PID29692) 2022-09-08-10:55:46.723.607 Start compiling 'Momentum.construct' and it will take a while. Please wait...
[EVENT] (PID29692) 2022-09-08-10:55:46.764.006 End compiling 'Momentum.construct'.
[EVENT] (PID29692) 2022-09-08-10:55:51.390.778 Start compiling 'LeNet.construct' and it will take a while. Please wait...
[EVENT] (PID29692) 2022-09-08-10:55:51.428.831 End compiling 'LeNet.construct'.
--------------------
Epoch:[0/10], Train loss:2.3203, Accuracy:0.113, Time:5.26s, 
--------------------
Epoch:[1/10], Train loss:0.0699, Accuracy:0.956, Time:4.84s, 
--------------------
Epoch:[2/10], Train loss:0.0157, Accuracy:0.976, Time:4.63s, 
--------------------
Epoch:[3/10], Train loss:0.0570, Accuracy:0.974, Time:4.54s, 
--------------------
Epoch:[4/10], Train loss:0.0255, Accuracy:0.9

训练过程中会打印loss值，loss值会波动，但总体来说loss值会逐步减小，精度逐步提高。每个人运行的loss值有一定随机性，不一定完全相同。